# 问题

- 你想给某个实例attribute 增加除访问与修改之外的其他处理逻辑，比如类型检查
或合法性验证。

## 解决方案

- 自定义某个属性的一种简单方法是将它定义为一个property。例如，下面的代码定
义了一个property，增加对一个属性简单的类型检查：

In [9]:
class Person:
    def __init__(self, first_name):
        self._first_name = first_name
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError("Expected a string")
        self._first_name = value
        
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute.")
        
        


- 上述代码中有三个相关联的方法，这三个方法的名字都必须一样。第一个方法是一
个getter 函数，它使得first_name 成为一个属性。其他两个方法给first_name 属
性添加了setter 和deleter 函数。需要强调的是只有在first_name 属性被创建后，
后面的两个装饰器@first_name.setter 和@first_name.deleter 才能被定义。
- property 的一个关键特征是它看上去跟普通的attribute 没什么两样，但是访问它
的时候会自动触发getter 、setter 和deleter 方法。例如：

In [5]:
p = Person("xxx")

print(p.first_name)

p.first_name = 111

xxx


TypeError: Expected a string

In [7]:
p.first_name = 'aaa'

In [10]:
del p.first_name

AttributeError: Can't delete attribute.

- 在实现一个property 的时候，底层数据(如果有的话) 仍然需要存储在某个地方。
因此，在get 和set 方法中，你会看到对_first_name 属性的操作，这也是实际数据
保存的地方。另外，你可能还会问为什么`__init__()` 方法中设置了self.first_name
而不是self._first_name 。在这个例子中，我们创建一个property 的目的就是在设置
attribute 的时候进行检查。因此，你可能想在初始化的时候也进行这种类型检查。通
过设置self.first_name ，自动调用setter 方法，这个方法里面会进行参数的检查，
否则就是直接访问self._first_name 了。
- 还能在已存在的get 和set 方法基础上定义property。例如：

In [11]:
class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)
        
    def get_first_name(self):
        return self._first_name
    
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError("Expected a string.")
        self._first_name = value
        
    def del_first_name(self):
        raise AttributeError("Can't delete attribute.")
        
    name = property(get_first_name, set_first_name, del_first_name)

In [13]:
p = Person('xxx')

p.name

'xxx'

In [15]:
p.name = 'aaa'
p.name

'aaa'

In [17]:
del p.name

AttributeError: Can't delete attribute.

## 讨论

- 一个property 属性其实就是一系列相关绑定方法的集合。如果你去查看拥有
property 的类，就会发现property 本身的fget、fset 和fdel 属性就是类里面的普通方
法。比如：

```python
>>> Person.first_name.fget
<function Person.first_name at 0x1006a60e0>
>>> Person.first_name.fset
<function Person.first_name at 0x1006a6170>
>>> Person.first_name.fdel
<function Person.first_name at 0x1006a62e0>
>>>
```

- 通常来讲，你不会直接取调用fget 或者fset，它们会在访问property 的时候自动
被触发。
- 只有当你确实需要对attribute 执行其他额外的操作的时候才应该使用到property。
有时候一些从其他编程语言(比如Java) 过来的程序员总认为所有访问都应该通过
getter 和setter，所以他们认为代码应该像下面这样写：

In [22]:
class Person:
    def __init__(self, first_name):
        self._first_name = first_name
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = value

- 不要写这种没有做任何其他额外操作的property。首先，它会让你的代码变得很臃
肿，并且还会迷惑阅读者。其次，它还会让你的程序运行起来变慢很多。最后，这样的设
计并没有带来任何的好处。特别是当你以后想给普通attribute 访问添加额外的处理逻
辑的时候，你可以将它变成一个property 而无需改变原来的代码。因为访问attribute
的代码还是保持原样。

- Properties 还是一种定义动态计算attribute 的方法。这种类型的attributes 并不会
被实际的存储，而是在需要的时候计算出来。比如：

In [23]:
import math

class Cirtle:
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def area(self):
        return math.pi * self.radius ** 2
    
    @property
    def diameter(self):
        return self.radius * 2
    
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

- 在这里，我们通过使用properties，将所有的访问接口形式统一起来，对半径、直
径、周长和面积的访问都是通过属性访问，就跟访问简单的attribute 是一样的。如果
不这样做的话，那么就要在代码中混合使用简单属性访问和方法调用。下面是使用的实
例：

In [25]:
c = Cirtle(4.0)

print(c.area)
print(c.diameter)
print(c.perimeter)

50.26548245743669
8.0
25.132741228718345


- 尽管properties 可以实现优雅的编程接口，但有些时候你还是会想直接使用getter
和setter 函数。例如：
```python
>>> p = Person('Guido')
>>> p.get_first_name()
'Guido'
>>> p.set_first_name('Larry')
>>>
```

- 这种情况的出现通常是因为Python 代码被集成到一个大型基础平台架构或程序
中。例如，有可能是一个Python 类准备加入到一个基于远程过程调用的大型分布式系
统中。这种情况下，直接使用get/set 方法(普通方法调用) 而不是property 或许会更
容易兼容。

- 最后一点，不要像下面这样写有大量重复代码的property 定义：

```python
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def first_name(self):
        return self._first_name
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    
    # Repeated property code, but for a different name (bad!)
    @property
    def last_name(self):
        return self._last_name
    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._last_name = value
```

- 重复代码会导致臃肿、易出错和丑陋的程序。好消息是，通过使用装饰器或闭包，
有很多种更好的方法来完成同样的事情。可以参考8.9 和9.21 小节的内容。